<a href="https://colab.research.google.com/github/vin00d/agentic-saul/blob/main/fine_tune_emb_and_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning Embeddings



## Task 1: Dependencies and Boilerplate

We'll set up our `nest_asyncio` so we can leverage async loops in our Notebook.

We'll also install the required libraries we'll be using today, and set up our OpenAI API key!

### 1. Import Libs

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
!pip install -qU langchain_openai langchain_huggingface langchain_core langchain langchain_community langchain-text-splitters pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.1/414.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

In [4]:
!pip install -qU faiss-cpu python-pptx==1.0.2 nltk==3.9.1 pymupdf beautifulsoup4 lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 14.9 MB/s eta 0:00:00


### OpenAI API Key

In [5]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

Enter Your OpenAI API Key: ··········


## 2. Load Data

In [6]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader#, UnstructuredEPubLoader
# from langchain_community.document_loaders import BSHTMLLoader


path = "data/"
# epub_loader = UnstructuredEPubLoader(path + "BlacksLaw9thEdition.epub")
# read all pdfs in the directory
pdf_loader = DirectoryLoader(path, glob="**/*.pdf", loader_cls=PyPDFLoader)

In [8]:
# epub_data = epub_loader.load()
pdf_data = pdf_loader.load()

In [9]:
pdf_data[0].page_content

'GLOSSARY\xa0‐\xa0COMMON\xa0LEGAL\xa0TERMS\nNOTE:\xa0\xa0The\xa0following\xa0definitions\xa0are\xa0not\xa0legal\xa0definitions.\xa0\xa0Rather,\xa0these\xa0definitions\xa0are\nintended\xa0to\xa0give\xa0you\xa0a\xa0general\xa0idea\xa0of\xa0the\xa0meanings\xa0of\xa0common\xa0legal\xa0words.\xa0\xa0For\xa0\ncomprehensive\xa0Definitions\xa0of\xa0legal\xa0terms,\xa0you\xa0may\xa0wish\xa0to\xa0consult\xa0a\xa0legal\xa0dictionary\n\xa0“Black’s\xa0Law\xa0Dictionary”\xa0is\xa0one\xa0such\xa0legal\xa0dictionary\xa0which\xa0is\xa0usually\xa0available\xa0at\n\xa0most\xa0law\xa0libraries.\nThis\xa0glossary\xa0of\xa0common\xa0legal\xa0terms\xa0is\xa0also\xa0available\xa0on‐line\xa0at:\nhttp://www.nycourts.gov/lawlibraries/glossary.shtml\n \nADDITIONAL\xa0ON‐LINE\xa0RESOURCES:\nhttp://www.nolo.com/glossary.cfm\xa0\nNolo’s\xa0on‐line\xa0legal\xa0dictionary.\nhttp://www.law‐dictionary.org/\nFree\xa0on‐line\xa0legal\xa0dictionary\xa0search\xa0engine.\nhttp://www.law.cornell.edu/wex\nLegal\xa0dictionary\x

Next, we'll set up a classic naive chunking strategy as we only care that the documents get parsed into chunks that we can generate synthetic questions about.

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 750,
    chunk_overlap  = 20,
    length_function = len
)

In [11]:
training_documents = text_splitter.split_documents(pdf_loader.load())

In [12]:
len(training_documents)

151

Next, we're going to associate each of our chunks with a unique identifier.

In [13]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

In [14]:
# create train, val and test splits
def split_data(data, train_size=0.7, val_size=0.15):
    train_end = int(len(data) * train_size)
    val_end = train_end + int(len(data) * val_size)
    train_data = data[:train_end]
    val_data = data[train_end:val_end]
    test_data = data[val_end:]
    return train_data, val_data, test_data
train_data, val_data, test_data = split_data(training_documents)

In [15]:
len(train_data), len(val_data), len(test_data)

(105, 22, 24)

## 3: Constructing a Fine-tuning Dataset

Using the nodes we created above, we can finally start constructing a fine-tuning dataset utilizing OpenAI's `gpt-4o-mini` (released [today](https://openai.com/index/gpt-4o-mini-advancing-cost-efficient-intelligence/)).

The basic idea here is straightforward enough:

1. We look at a document
2. We generate questions that could be answered by that node

This gives us a number of question/context pairs that we can use to fine-tune our Embeddings model.

In [16]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

We'll create a simple Question Generation prompt to query `gpt-4o-mini` to generate Questions for each retrieved context.

In [17]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

We'll create a simple chain to query the LLM!

In [18]:
question_generation_chain = qa_prompt_template | qa_chat_model

In [19]:
tmp_response = question_generation_chain.invoke({"n_questions": 5, "context": training_documents[0].page_content})

In [20]:
# parse response
import re
def parse_response(response):
    questions = re.findall(r"\d+\.\s*(.*)", response)
    return questions
questions = parse_response(tmp_response.content)

In [21]:
questions

['What is the purpose of the glossary of common legal terms provided in the context?',
 'Are the definitions in the glossary considered legal definitions?',
 'Which legal dictionary is mentioned as a resource for comprehensive definitions of legal terms?',
 'Where can one find the glossary of common legal terms online?',
 'What additional online resources are provided for legal dictionary searches?']

In [22]:
import tqdm
import re

async def create_questions(documents, n_questions):
  questions = {}
  relevant_docs = {}

  def _parse_response(response):
    questions = re.findall(r"\d+\.\s*(.*)", response)
    return questions

  for document in tqdm.tqdm(documents):
    # generate questions
    response = await question_generation_chain.ainvoke(
      input={"context": document.page_content, "n_questions": n_questions}
    )
    # parse response and get questions_list
    questions_list = _parse_response(response.content)

    for question in questions_list:
      # create a unique id for the question
      question_id = str(uuid.uuid4())
      questions[question_id] = question  # question_id : question
      relevant_docs[question_id] = [document.metadata["id"]] # question_id : document.id

  return questions, relevant_docs

In [23]:
tmp_qs, tmp_contexts = await create_questions(train_data[:3], 2)

100%|██████████| 3/3 [00:02<00:00,  1.38it/s]


In [24]:
tmp_qs

{'0274bcab-c674-4200-9288-75f970081b09': 'What is the purpose of the glossary of common legal terms provided in the context?  ',
 '660239e1-7c82-48f1-bd5c-85b591a09eef': 'Where can one find a comprehensive legal dictionary for more detailed definitions of legal terms?',
 'b50d5c57-c18d-425d-8ec8-f9f3acda595e': 'What organization maintains the legal dictionary and encyclopedia mentioned in the context?  ',
 '455bbe80-6c2c-43f0-9552-30904a66ba53': 'Where is the Legal Information Institute located?',
 '3ebb9b55-a41d-44e3-b90f-75aa92f4cf10': 'What is the purpose of an acknowledgment in a legal context?  ',
 '27bfdc89-6e4c-40af-a186-53798aedb5ce': 'How does the act of acquitting someone affect their legal status regarding the charges against them?'}

In [25]:
tmp_contexts

{'0274bcab-c674-4200-9288-75f970081b09': ['24308820-375a-4b0b-8e69-63a47582a393'],
 '660239e1-7c82-48f1-bd5c-85b591a09eef': ['24308820-375a-4b0b-8e69-63a47582a393'],
 'b50d5c57-c18d-425d-8ec8-f9f3acda595e': ['eeb94b78-72d2-4dca-9d47-b75a31573843'],
 '455bbe80-6c2c-43f0-9552-30904a66ba53': ['eeb94b78-72d2-4dca-9d47-b75a31573843'],
 '3ebb9b55-a41d-44e3-b90f-75aa92f4cf10': ['6af1f5cf-2f1b-4896-8d6e-002c80f40d17'],
 '27bfdc89-6e4c-40af-a186-53798aedb5ce': ['6af1f5cf-2f1b-4896-8d6e-002c80f40d17']}

In [27]:
training_questions, training_relevant_contexts = await create_questions(train_data, 2)

100%|██████████| 105/105 [01:19<00:00,  1.33it/s]


In [28]:
print("Number of training questions:", len(training_questions))
print("Number of documents:", len(train_data))

Number of training questions: 210
Number of documents: 105


We'll use the function to generate validation, and test data.

In [30]:
val_questions, val_relevant_contexts = await create_questions(val_data, 2)

100%|██████████| 22/22 [00:20<00:00,  1.10it/s]


In [31]:
test_questions, test_relevant_contexts = await create_questions(test_data, 2)

100%|██████████| 24/24 [00:22<00:00,  1.09it/s]


In [33]:
print(f"Val :: questions: {len(val_questions)}, documents: {len(val_data)}")
print(f"Test :: questions: {len(test_questions)}, documents: {len(test_data)}")

Val :: questions: 44, documents: 22
Test :: questions: 48, documents: 24


### Reformating and Saving Datasets

Now, we can save our datasets for later use!

In [35]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in train_data}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open("data/training_dataset.jsonl", "w") as f:
  json.dump(train_dataset, f)

In [36]:
val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_data}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open("data/val_dataset.jsonl", "w") as f:
  json.dump(val_dataset, f)

In [37]:
test_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_data}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : test_corpus
}

with open("data/test_dataset.jsonl", "w") as f:
  json.dump(test_dataset, f)

In [38]:
def print_lengths(dataset):
    for key, value in dataset.items():
        print(key, len(value))

In [39]:
print_lengths(train_dataset)
print_lengths(val_dataset)
print_lengths(test_dataset)

questions 210
relevant_contexts 210
corpus 105
questions 44
relevant_contexts 44
corpus 22
questions 48
relevant_contexts 48
corpus 24


## 4: Fine-tuning `snowflake-arctic-embed-l`

- Getting the `sentence-transformers` Embeddings model [`snowflake-arctic-embed-l`](https://huggingface.co/Snowflake/snowflake-arctic-embed-l) as a base embeddings model.

In [41]:
!pip install -qU sentence_transformers datasets "pyarrow<19.0.0a0,>=14.0.0"

In [42]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-l"
model = SentenceTransformer(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/85.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

- imports from `sentence_transformers` and `torch`.

In [43]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

> NOTE: Best to use a much larger batch size (~64+), hardware permitting.

In [44]:
BATCH_SIZE = 10

Let's move our dataset into the expected format for training.

In [45]:
# first load the dataset from the jsonl files

import json
def load_dataset(file_path):
    with open(file_path, "r") as f:
        dataset = json.load(f)
    return dataset

In [46]:
train_dataset = load_dataset("data/training_dataset.jsonl")
val_dataset = load_dataset("data/val_dataset.jsonl")
test_dataset = load_dataset("data/test_dataset.jsonl")

In [47]:
print("\ntrain_dataset: ")
print_lengths(train_dataset)

print("\nval_dataset: ")
print_lengths(val_dataset)

print("\ntest_dataset: ")
print_lengths(test_dataset)


train_dataset: 
questions 210
relevant_contexts 210
corpus 105

val_dataset: 
questions 44
relevant_contexts 44
corpus 22

test_dataset: 
questions 48
relevant_contexts 48
corpus 24


In [48]:
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

Now we can create a `torch` `DataLoader`!

In [49]:
# examples[0].texts

In [50]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

### Loss Function

- `MultipleNegativesRankingLoss` - more information [here](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/MultipleNegativesRankingLoss.py).
- "Wrapped" in `MatryoshkaLoss` - more information [here](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/MatryoshkaLoss.py).

In [51]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

### Set-up Evaluator

In [52]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [53]:
EPOCHS = 10

### Training Setup
> NOTE: We're manually defining a warm-up period here - this is just to provide a smooth ramp into our training!

In [54]:
# get wandb api key
wandb_api_key = getpass.getpass("Enter your wandb api key: ")

Enter your wandb api key: ··········


In [55]:
import wandb
wandb.login(key=wandb_api_key)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vinod to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [56]:
import wandb
wandb.init(
    project="emb-model-ft-agentic-saul",
    name="Snowflake-arctic-embed-l",
    config={
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "model_id": model_id
    }
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [57]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='finetuned_arctic_ft',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
21,No log,No log,0.840909,0.977273,0.977273,0.977273,0.840909,0.325758,0.195455,0.097727,0.840909,0.977273,0.977273,0.977273,0.923969,0.905303,0.907197
42,No log,No log,0.931818,0.954545,0.977273,1.000000,0.931818,0.318182,0.195455,0.100000,0.931818,0.954545,0.977273,1.000000,0.962787,0.951389,0.951389
50,No log,No log,0.931818,0.954545,0.977273,1.000000,0.931818,0.318182,0.195455,0.100000,0.931818,0.954545,0.977273,1.000000,0.962787,0.951389,0.951389
63,No log,No log,0.909091,0.931818,0.977273,1.000000,0.909091,0.310606,0.195455,0.100000,0.909091,0.931818,0.977273,1.000000,0.950176,0.934659,0.934659
84,No log,No log,0.931818,0.931818,0.954545,1.000000,0.931818,0.310606,0.190909,0.100000,0.931818,0.931818,0.954545,1.000000,0.956872,0.944129,0.944129
100,No log,No log,0.931818,0.931818,0.954545,1.000000,0.931818,0.310606,0.190909,0.100000,0.931818,0.931818,0.954545,1.000000,0.956272,0.943561,0.943561
105,No log,No log,0.931818,0.931818,0.954545,1.000000,0.931818,0.310606,0.190909,0.100000,0.931818,0.931818,0.954545,1.000000,0.955618,0.942866,0.942866
126,No log,No log,0.931818,0.931818,0.954545,1.000000,0.931818,0.310606,0.190909,0.100000,0.931818,0.931818,0.954545,1.000000,0.956872,0.944129,0.944129
147,No log,No log,0.931818,0.931818,0.954545,1.000000,0.931818,0.310606,0.190909,0.100000,0.931818,0.931818,0.954545,1.000000,0.955547,0.942677,0.942677
150,No log,No log,0.931818,0.931818,0.954545,1.000000,0.931818,0.310606,0.190909,0.100000,0.931818,0.931818,0.954545,1.000000,0.955547,0.942677,0.942677


In [58]:
from huggingface_hub import notebook_login

notebook_login()

In [59]:
hf_username = "vin00d"

In [60]:
model.push_to_hub(f"{hf_username}/snowflake-arctic-legal-ft-1")

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

'https://huggingface.co/vin00d/snowflake-arctic-legal-ft-1/commit/d09a7b51a3f68ee1d73fe2e4cc930d521e56a057'

## 5: Evaluating our Retriever

In [61]:
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

Now we'll define a function that will help us evaluate our retrieval process.

> NOTE: We're assuming 1 correct document in a "hit".

In [62]:
import tqdm

def evaluate_openai(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
  corpus = dataset['corpus']
  questions = dataset['questions']
  relevant_docs = dataset['relevant_contexts']
  documents = [Document(page_content=content, metadata={"id": doc_id}) for doc_id, content in corpus.items()]
  vectorstore = FAISS.from_documents(documents, embed_model)

  retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

  eval_results = []
  for id, question in tqdm.tqdm(questions.items()):
    retrieved_nodes = retriever.invoke(question)
    retrieved_ids = [node.metadata["id"] for node in retrieved_nodes]
    expected_id = relevant_docs[id][0]
    is_hit = expected_id in retrieved_ids
    eval_results.append({"id": id, "question": question, "expected_id": expected_id, "is_hit": is_hit})

  return eval_results

Evaluating our model against:

1. OpenAI's closed source `text-embedding-3-small`
2. The base non-fine-tuned version of `Snowflake/snowflake-arctic-embed-l`.

### `text-embedding-3-small`

In [63]:
te3_openai = OpenAIEmbeddings(model="text-embedding-3-small")
te3_results = evaluate_openai(test_dataset, te3_openai)

100%|██████████| 48/48 [00:29<00:00,  1.62it/s]


In [64]:
te3_results_df = pd.DataFrame(te3_results)

In [65]:
te3_hit_rate = te3_results_df["is_hit"].mean()
te3_hit_rate

1.0

### `Snowflake/snowflake-arctic-embed-l` (base)

In [66]:
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-l")
arctic_embed_m_results = evaluate_openai(test_dataset, huggingface_embeddings)

100%|██████████| 48/48 [00:01<00:00, 47.48it/s]


In [67]:
arctic_embed_m_results_df = pd.DataFrame(arctic_embed_m_results)

In [68]:
arctic_embed_m_hit_rate = arctic_embed_m_results_df["is_hit"].mean()
arctic_embed_m_hit_rate

0.875

### `Snowflake/snowflake-arctic-embed-l` (fine-tuned)

In [69]:
finetune_embeddings = HuggingFaceEmbeddings(model_name="finetuned_arctic_ft")
finetune_results = evaluate_openai(test_dataset, finetune_embeddings)

Some weights of BertModel were not initialized from the model checkpoint at finetuned_arctic_ft and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 48/48 [00:00<00:00, 48.31it/s]


In [70]:
finetune_results_df = pd.DataFrame(finetune_results)

In [71]:
finetune_hit_rate = finetune_results_df["is_hit"].mean()
finetune_hit_rate

1.0

# Evaluating RAG Pipeline

## 1. Vibe Checking the RAG Pipeline

We're going to use our RAG pipeline to vibe check on some common phrases now that we've modified it!

### Creating New Chunks

In order to try and evaluate our system more fairly, let's create new chunks that we will use to create our Vector Store.

In [72]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap  = 50,
    length_function = len
)

training_documents = text_splitter.split_documents(pdf_loader.load())

### Base Chain

We'll start by constructing our base chain, which will use the untrained retrieval model.

#### R - Retrieval

In [73]:
from langchain_community.vectorstores import FAISS

base_vectorstore = FAISS.from_documents(training_documents, huggingface_embeddings)
base_retriever = base_vectorstore.as_retriever(search_kwargs={"k": 6})

#### A - Augmented

In [74]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and a question, you must answer the question. If you do not know the answer, you must state that you do not know.

Context:
{context}

Question:
{question}

Answer:
"""

rag_prompt_template = ChatPromptTemplate.from_template(RAG_PROMPT)

#### G - Generation

In [75]:
from langchain_openai import ChatOpenAI

rag_llm =  ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

#### RAG - LCEL RAG Pipeline

In [76]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

base_rag_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [77]:
base_rag_chain.invoke({"question" : "What is an Amicus curiae?"})["response"]

'An amicus curiae, or "friend of the court," is a person or organization that is not a party to a legal case but offers information, expertise, or insight that may assist the court in making its decision. This can include submitting briefs or participating in oral arguments to provide additional perspectives on the issues at hand.'

In [78]:
base_rag_chain.invoke({"question" : "What is Sua Sponte?"})["response"]

'I do not know.'

In [80]:
base_rag_chain.invoke({"question" : "What is Article III judge?"})["response"]

'I do not know.'

In [81]:
base_rag_chain.invoke({"question" : "What is Nolo contendere?"})["response"]

'I do not know.'

### Fine-tuned Embedding Model

Now let's rebuild our RAG chain with the Fine-tuned model - the only component we need to change is our `FAISS` vectorstore!

In [82]:
finetune_vectorstore = FAISS.from_documents(training_documents, finetune_embeddings)
finetune_retriever = finetune_vectorstore.as_retriever(search_kwargs={"k": 6})

In [83]:
finetune_rag_chain = (
    {"context": itemgetter("question") | finetune_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [84]:
finetune_rag_chain.invoke({"question" : "What is an Amicus curiae?"})["response"]

'An Amicus curiae is a Latin term meaning "friend of the court." It refers to advice formally offered to the court in a brief filed by an entity that is interested in, but not a party to, the case.'

In [85]:
finetune_rag_chain.invoke({"question" : "What is Sua Sponte?"})["response"]

'Sua Sponte is a Latin term meaning "of one\'s own accord." It refers to a court taking an action in a case without being asked to do so by either side.'

In [86]:
finetune_rag_chain.invoke({"question" : "What is Article III judge?"})["response"]

'An Article III judge is a federal judge who is appointed for life, during "good behavior," under Article III of the Constitution. These judges are nominated by the President and confirmed by the Senate.'

In [87]:
finetune_rag_chain.invoke({"question" : "What is Nolo contendere?"})["response"]

'Nolo contendere, or "no contest," is a plea in a criminal case where the defendant does not admit guilt but accepts the same consequences as a guilty plea regarding the criminal sentence. However, it may not be considered an admission of guilt for any other purpose.'

## 2. RAGAS Evaluation

Going beyond vibe checks.

In [88]:
!pip install -qU ragas==0.2.10 unstructured==0.16.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 12.0 MB/s eta 0:00:00


In [93]:
!pip install -qU "unstructured[pdf]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pdfplumber 0.11.5 requires pdfminer.six==20231228, but you have pdfminer-six 20240706 which is incompatible.


In [89]:
os.environ["RAGAS_APP_TOKEN"] = getpass.getpass("Please enter your Ragas API key!")

Please enter your Ragas API key!··········


In [94]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

### Golden Test Data Set

In [95]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.pdf")
docs = loader.load()

In [96]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/19 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/40 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [97]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Where can I find Nolo's online legal dictionary?,[GLOSSARY ‐ COMMON LEGAL TERMS NOTE: The follo...,Nolo's online legal dictionary can be found at...,single_hop_specifc_query_synthesizer
1,What role does an Appellate Court play in the ...,[A- Acknowledgment: The act of going before a ...,The Appellate Court is responsible for reviewi...,single_hop_specifc_query_synthesizer
2,what calendar call mean?,[C- Calendar: A schedule of matters to be hear...,Calendar Call: The calling of matters requirin...,single_hop_specifc_query_synthesizer
3,What does 'Dismissal with Prejudice' mean in l...,[D- Damages: Monetary compensation for wrong o...,Dismissal with Prejudice refers to an action d...,single_hop_specifc_query_synthesizer
4,How do court procedures ensure the future appe...,[<1-hop>\n\nB- Bail: The security given (or po...,Court procedures ensure the future appearance ...,multi_hop_abstract_query_synthesizer
5,How do court procedures ensure the future appe...,[<1-hop>\n\nB- Bail: The security given (or po...,Court procedures ensure the future appearance ...,multi_hop_abstract_query_synthesizer
6,"What is the role of bail in court procedures, ...",[<1-hop>\n\nB- Bail: The security given (or po...,Bail serves as a security given or posted to e...,multi_hop_abstract_query_synthesizer
7,How do legal documents such as a complaint and...,[<1-hop>\n\nC- Calendar: A schedule of matters...,"In a court case, legal documents play a crucia...",multi_hop_abstract_query_synthesizer
8,What role do U.S. district courts play in the ...,[<1-hop>\n\nDR) A procedure for settling a dis...,U.S. district courts serve as the trial courts...,multi_hop_specific_query_synthesizer
9,What are the key differences between Chapter 9...,"[<1-hop>\n\ndebtor must satisfy a ""means test....",Chapter 9 of the Bankruptcy Code provides for ...,multi_hop_specific_query_synthesizer


In [98]:
dataset.upload()

Testset uploaded! View at https://app.ragas.io/dashboard/alignment/testset/16940761-8548-47a4-af8f-c5086c8b1715


'https://app.ragas.io/dashboard/alignment/testset/16940761-8548-47a4-af8f-c5086c8b1715'

### Eval - Base Embedding Model - `base_rag_chain`

In [99]:
for test_row in dataset:
  response = base_rag_chain.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [100]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,Where can I find Nolo's online legal dictionary?,[http://www.nolo.com/glossary.cfm \nNolo’s on‐...,[GLOSSARY ‐ COMMON LEGAL TERMS NOTE: The follo...,You can find Nolo's online legal dictionary at...,Nolo's online legal dictionary can be found at...,single_hop_specifc_query_synthesizer
1,What role does an Appellate Court play in the ...,[The party who appeals a district court's deci...,[A- Acknowledgment: The act of going before a ...,An Appellate Court has the power to review the...,The Appellate Court is responsible for reviewi...,single_hop_specifc_query_synthesizer
2,what calendar call mean?,[an arrest.\nWitness\nA person called upon by ...,[C- Calendar: A schedule of matters to be hear...,I do not know.,Calendar Call: The calling of matters requirin...,single_hop_specifc_query_synthesizer
3,What does 'Dismissal with Prejudice' mean in l...,"[during some other formal conversation, such a...",[D- Damages: Monetary compensation for wrong o...,I do not know.,Dismissal with Prejudice refers to an action d...,single_hop_specifc_query_synthesizer
4,How do court procedures ensure the future appe...,"[about the case., during some other formal con...",[<1-hop>\n\nB- Bail: The security given (or po...,I do not know.,Court procedures ensure the future appearance ...,multi_hop_abstract_query_synthesizer
5,How do court procedures ensure the future appe...,"[during some other formal conversation, such a...",[<1-hop>\n\nB- Bail: The security given (or po...,I do not know.,Court procedures ensure the future appearance ...,multi_hop_abstract_query_synthesizer
6,"What is the role of bail in court procedures, ...","[during some other formal conversation, such a...",[<1-hop>\n\nB- Bail: The security given (or po...,I do not know.,Bail serves as a security given or posted to e...,multi_hop_abstract_query_synthesizer
7,How do legal documents such as a complaint and...,"[during some other formal conversation, such a...",[<1-hop>\n\nC- Calendar: A schedule of matters...,I do not know.,"In a court case, legal documents play a crucia...",multi_hop_abstract_query_synthesizer
8,What role do U.S. district courts play in the ...,"[do something (e.g., injunction). In American ...",[<1-hop>\n\nDR) A procedure for settling a dis...,I do not know.,U.S. district courts serve as the trial courts...,multi_hop_specific_query_synthesizer
9,What are the key differences between Chapter 9...,[trustee may also bring actions against credit...,"[<1-hop>\n\ndebtor must satisfy a ""means test....",I do not know.,Chapter 9 of the Bankruptcy Code provides for ...,multi_hop_specific_query_synthesizer


In [101]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

In [102]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

In [103]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[14]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
ERROR:ragas.executor:Exception raised in Job[20]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
ERROR:ragas.executor:Exception raised in Job[32]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
ERROR:ragas.executor:Exception raised in Job[38]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe'')
ERROR:ragas.executor:Exception raised in Job[26]: TypeError(ufunc 'invert' not supported for the input types, and the inputs could not b

{'context_recall': 0.2583, 'faithfulness': 0.3967, 'factual_correctness': 0.6225, 'answer_relevancy': 0.3189, 'context_entity_recall': 0.2639, 'noise_sensitivity_relevant': 0.0354}

### Eval - FineTuned Embedding Model - `finetune_rag_chain`

In [104]:
for test_row in dataset:
  response = finetune_rag_chain.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [105]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,Where can I find Nolo's online legal dictionary?,[http://www.nolo.com/glossary.cfm \nNolo’s on‐...,[GLOSSARY ‐ COMMON LEGAL TERMS NOTE: The follo...,You can find Nolo's online legal dictionary at...,Nolo's online legal dictionary can be found at...,single_hop_specifc_query_synthesizer
1,What role does an Appellate Court play in the ...,[The party who appeals a district court's deci...,[A- Acknowledgment: The act of going before a ...,An Appellate Court has the power to review the...,The Appellate Court is responsible for reviewi...,single_hop_specifc_query_synthesizer
2,what calendar call mean?,[Brief: A written or printed document prepare...,[C- Calendar: A schedule of matters to be hear...,A Calendar Call is the calling of matters requ...,Calendar Call: The calling of matters requirin...,single_hop_specifc_query_synthesizer
3,What does 'Dismissal with Prejudice' mean in l...,[A debt for which the Bankruptcy Code allows t...,[D- Damages: Monetary compensation for wrong o...,'Dismissal with Prejudice' means that the cour...,Dismissal with Prejudice refers to an action d...,single_hop_specifc_query_synthesizer
4,How do court procedures ensure the future appe...,"[Property of all kinds, including real and per...",[<1-hop>\n\nB- Bail: The security given (or po...,Court procedures ensure the future appearance ...,Court procedures ensure the future appearance ...,multi_hop_abstract_query_synthesizer
5,How do court procedures ensure the future appe...,"[Property of all kinds, including real and per...",[<1-hop>\n\nB- Bail: The security given (or po...,Court procedures ensure the future appearance ...,Court procedures ensure the future appearance ...,multi_hop_abstract_query_synthesizer
6,"What is the role of bail in court procedures, ...","[Property of all kinds, including real and per...",[<1-hop>\n\nB- Bail: The security given (or po...,Bail serves as a financial guarantee that a de...,Bail serves as a security given or posted to e...,multi_hop_abstract_query_synthesizer
7,How do legal documents such as a complaint and...,"[-4-\nCaption: In a pleading, deposition or o...",[<1-hop>\n\nC- Calendar: A schedule of matters...,Legal documents such as a complaint and a cert...,"In a court case, legal documents play a crucia...",multi_hop_abstract_query_synthesizer
8,What role do U.S. district courts play in the ...,[The party who appeals a district court's deci...,[<1-hop>\n\nDR) A procedure for settling a dis...,U.S. district courts serve as the trial courts...,U.S. district courts serve as the trial courts...,multi_hop_specific_query_synthesizer
9,What are the key differences between Chapter 9...,[A legal claim.\nChambers\nThe offices of a ju...,"[<1-hop>\n\ndebtor must satisfy a ""means test....",The provided context does not contain specific...,Chapter 9 of the Bankruptcy Code provides for ...,multi_hop_specific_query_synthesizer


In [106]:
evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

In [107]:
custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/72 [00:00<?, ?it/s]

{'context_recall': 0.7667, 'faithfulness': 0.7221, 'factual_correctness': 0.6925, 'answer_relevancy': 0.8779, 'context_entity_recall': 0.3065, 'noise_sensitivity_relevant': 0.1734}

### Results

**Base Embedding Model**
```python
{'context_recall': 0.2583, 'faithfulness': 0.3967, 'factual_correctness': 0.6225, 'answer_relevancy': 0.3189, 'context_entity_recall': 0.2639, 'noise_sensitivity_relevant': 0.0354}
```

**Finetuned Embedding Model**
```python
{'context_recall': 0.7667, 'faithfulness': 0.7221, 'factual_correctness': 0.6925, 'answer_relevancy': 0.8779, 'context_entity_recall': 0.3065, 'noise_sensitivity_relevant': 0.1734}
```

Performance went up on all metrics after fine tuning!!